In [1]:
%load_ext autoreload

%autoreload 2

In [8]:
%reload_ext autoreload

In [9]:
!pip install wget
!python -m wget https://chromedriver.storage.googleapis.com/108.0.5359.71/chromedriver_win32.zip

You should consider upgrading via the 'C:\Users\user\.virtualenvs\google-arts-0o2XoGZ3\Scripts\python.exe -m pip install --upgrade pip' command.



Saved under chromedriver_win32.zip


In [2]:
from src.scraping.scraping_section import ScrapingSection
import time
path_driver = 'chromedriver.exe'
scraper = ScrapingSection(path_driver)
scraper.open("WHITE")
scraper.scroll_down(0)
#time.sleep(2)
scraper.scrape()
#print(src)

Open page section WHITE
Finish scrolling down
[{'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM_TtHnpV4uxifY2CR4N_7aK3dNuIQaNCvCLymW8qlht_f4w6RxNpwLJxcGe94_hKVdNJvfrHMQ=w313-c-h313-rw-v1")', 'page': 'https://artsandculture.google.com/asset/the-magpie/rQGnadHwK8lSmg'}, {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM8E27-JPnDd0MS2xdUy025M8a6HVoePPU8qxJ4FRcq--XVPtlKhgluQxwg501cHDCMnkdF21C4=w313-c-h313-fcrop64=1,00000d37ffff8ce4-rw-v1")', 'page': 'https://artsandculture.google.com/asset/symphony-in-white-no-1-the-white-girl/egEsaSX979J8mw'}, {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM9a9ZIa8v51w_ibyeqr53ek7YslNn9gRH35L83X9aB30hIil5aUE98Wgeng24RI5bNdnc7m5VY=w313-c-h313-fcrop64=1,00000b77ffffdd97-rw-v1")', 'page': 'https://artsandculture.google.com/asset/the-cradle/WAE_4Ek4Dn6EnA'}, {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM_xbsnsclF0ovIA8Llr8Uo8ZzEmsutG2SEed6Fl5FwvMMySsNpwwfj5GgjYPcyBXXBW3GAlw2g=w313-c-h313-fcrop64=1,3d7a0000fe28ffff-r

In [6]:

script_test:str = """
    const script = () =>{
        const containers = [...document.querySelectorAll('.DuHQbc')];
        const img_elements= containers.map(contain => contain.firstElementChild);
        return img_elements.map(img =>({
            page:img.href,
            image: window.getComputedStyle(img, false).backgroundImage
        }))  
    };
    return script()
    """

scraper.driver.execute_script(script_test)

[{'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM_TtHnpV4uxifY2CR4N_7aK3dNuIQaNCvCLymW8qlht_f4w6RxNpwLJxcGe94_hKVdNJvfrHMQ=w313-c-h313-rw-v1")',
  'page': 'https://artsandculture.google.com/asset/the-magpie/rQGnadHwK8lSmg'},
 {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM8E27-JPnDd0MS2xdUy025M8a6HVoePPU8qxJ4FRcq--XVPtlKhgluQxwg501cHDCMnkdF21C4=w313-c-h313-fcrop64=1,00000d37ffff8ce4-rw-v1")',
  'page': 'https://artsandculture.google.com/asset/symphony-in-white-no-1-the-white-girl/egEsaSX979J8mw'},
 {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM9a9ZIa8v51w_ibyeqr53ek7YslNn9gRH35L83X9aB30hIil5aUE98Wgeng24RI5bNdnc7m5VY=w313-c-h313-fcrop64=1,00000b77ffffdd97-rw-v1")',
  'page': 'https://artsandculture.google.com/asset/the-cradle/WAE_4Ek4Dn6EnA'},
 {'image': 'url("https://lh3.googleusercontent.com/ci/AC_FhM_xbsnsclF0ovIA8Llr8Uo8ZzEmsutG2SEed6Fl5FwvMMySsNpwwfj5GgjYPcyBXXBW3GAlw2g=w313-c-h313-fcrop64=1,3d7a0000fe28ffff-rw-v1")',
  'page': 'https://artsandcu

In [10]:
from src.scraping.scripts import scripts
scripts.script_test

"\n    cosnt script = () =>{\n        const containers = [...document.querySelectorAll('.DuHQbc')];\n        const img_elements= containers.map(contain => contain.firstElementChild);\n        return img_elements.map(img =>({\n            page:img.href,\n            image: window.getComputedStyle(img, false).backgroundImage\n        }))  \n    };\n    return script()\n    "

In [3]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path = path_driver)

C:\Users\user\AppData\Local\Temp\ipykernel_6948\4033232118.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = path_driver)
